In [ ]:
library(Seurat)
library(dplyr)
library(ggplot2)
library(viridis)
source("../Scripts/functions.R")
source("../Scripts/plots.r")
options(repr.plot.width=10, repr.plot.height=10)

# Get Figure 2 an

In [ ]:
umap <- as.data.frame(data.table::fread("../Results/umap_PBMC.csv",sep = ",", verbose = F))
tech_full <- read.csv("../Data/raw/PBMC/meta.txt", sep="\t")
file_train <- "../Data/processed/PBMC_Mosaic/data_train.txt"
file_test <-  "../Data/processed/PBMC_mono/data_test.txt"
data_train <- readData(file_train)
data_test  <- readData(file_test)
output <- "../Results/"

In [ ]:
tech <- tech_full[,c("NAME", "Method","CellType", "Experiment")]
tech = tech[-1,]
colnames(tech) <- c("id", "tech", "class", "experiment")
rownames(tech) <- tech$id
tech$set <- interaction(tech$experiment, tech$tech)

tech$type[tech$id %in% colnames(data_train)] <- "train"
tech$type[tech$id %in% colnames(data_test)] <- "test"
tech <- tech[!(is.na(tech$type)),]

umap[umap==TRUE]<-1
umap[is.na(umap)]<- 0

In [ ]:
predictions <- merge(umap, tech, by="id", all.y = T)
rownames(predictions) <- predictions$id
methods <- c("Seurat", "SCN", "CellID", "ItClust", "MLP", "SingleR")
versions <- c("Mosaic", "Mono")

for(method in methods){
    for(version in versions){
        if(version=="Mono") size = 100
        if(version=="Mosaic") size =136
        predictions <- get_percentage_predictions(predictions,version, method, size)
    }
}

In [ ]:
pbmc.integrated <- prepare_umap(file_train = file_train,file_test = file_test, meta_data = predictions, split = "set")

In [ ]:
cells <- predictions$id[predictions$type=="test"]
f2A <- plot_umap(pbmc.integrated, c("class"), NULL, "Spectral",1,1, cells )
f2A

In [ ]:
cells_train <- predictions$id[predictions$type=="train"]
f2B <- plot_umap(pbmc.integrated, c("class"), NULL, "Spectral",1,1, cells_train )
f2B

In [ ]:
f2AB <- ggpubr::ggarrange(f2A,f2B, common.legend = T, nrow=1, labels=c("A", "B"),font.label = list(size = 8))
f2AB

In [ ]:
leg <- ggpubr::get_legend(FeaturePlot(pbmc.integrated, reduction="umap", features=paste0("Mono_Seurat"),
                                      cols=viridis(101), cells = cells) + 
                        theme(legend.text=element_text(size=4), legend.position="bottom", legend.title=element_text(size=6),
                              rect = element_rect(fill = "white")) +
                          labs(color="Percentage of correct predictions"))



predictionUMAPs <-lapply(methods, function(method)plot_features(pbmc.integrated, method,cells, c("Mono_", "Mosaic_"), c("Mono", "Mosaic")))
names(predictionUMAPs) <- methods                      
                  
f2CG <- ggpubr::ggarrange(ggpubr::ggarrange(plotlist=c(predictionUMAPs, leg), ncol=2,nrow=3,
                                                           labels=c("C) Seurat", "D) SCN", "E) CellID", "F) ItClust", "G) MLP "),
                                                           font.label = list(size = 8)))     
#ggplot2::ggsave(paste0(output, "f3_mosaic.png"), plot=predictionUMAPsPlot, device = "png",
#                dpi=350, width=178, height = 160,units = "mm",bg = "white")
f2CG

In [ ]:
f2 <- ggpubr::ggarrange(f2AB,f2CG, common.legend = F, nrow=2, height=c(1,3),font.label = list(size = 8))
ggplot2::ggsave(paste0(output, "f2_mosaic.png"), plot=f2, device = "png", dpi=350, width=178, height = 160,units = "mm",bg = "white")
f2